In [9]:
jpegimages_path = "datasets/DAVIS_dataset/JPEGImages"

In [11]:
import os

jpegimages_path = "datasets/DAVIS_dataset/JPEGImages/480p"
all_dirs = [name for name in os.listdir(jpegimages_path) if os.path.isdir(os.path.join(jpegimages_path, name))]

print("🗂 Available JPEGImages Sequences:")
for d in sorted(all_dirs):
    print(f"• {d}")

🗂 Available JPEGImages Sequences:
• bear
• bike-packing
• blackswan
• bmx-bumps
• bmx-trees
• boat
• boxing-fisheye
• breakdance
• breakdance-flare
• bus
• camel
• car-roundabout
• car-shadow
• car-turn
• cat-girl
• classic-car
• color-run
• cows
• crossing
• dance-jump
• dance-twirl
• dancing
• disc-jockey
• dog
• dog-agility
• dog-gooses
• dogs-jump
• dogs-scale
• drift-chicane
• drift-straight
• drift-turn
• drone
• elephant
• flamingo
• goat
• gold-fish
• hike
• hockey
• horsejump-high
• horsejump-low
• india
• judo
• kid-football
• kite-surf
• kite-walk
• koala
• lab-coat
• lady-running
• libby
• lindy-hop
• loading
• longboard
• lucia
• mallard-fly
• mallard-water
• mbike-trick
• miami-surf
• motocross-bumps
• motocross-jump
• motorbike
• night-race
• paragliding
• paragliding-launch
• parkour
• pigs
• planes-water
• rallye
• rhino
• rollerblade
• schoolgirls
• scooter-black
• scooter-board
• scooter-gray
• sheep
• shooting
• skate-park
• snowboard
• soapbox
• soccerball
• stroll

In [35]:
import os
import json
from pathlib import Path

# === Paths ===
image_root = Path("datasets/DAVIS_dataset/PNGImages/480p")
single_json_path = Path("output/single_object_boundingboxes_labels.json")
multi_json_path = Path("output/multi_object_boundingboxes_labels.json")

# === Load annotation data ===
with open(single_json_path) as f:
    single_data = json.load(f)
with open(multi_json_path) as f:
    multi_data = json.load(f)

def check_missing_frames(data, image_root, dataset_name):
    missing = []

    for seq_name, frames in data.items():
        seq_path = image_root / seq_name
        for frame in frames.keys():
            image_file = seq_path / frame  # Use the frame name *as-is*
            if not image_file.exists():
                missing.append(f"{seq_name}/{frame}")

    # === Output Results ===
    print(f"\n🖼 Checking Missing Frames — {dataset_name}")
    if missing:
        print("❌ Missing Frame Images:")
        for m in missing:
            print(f"• {m}")
    else:
        print("✅ All raw image frame files are present.")

# === Run for both datasets ===
check_missing_frames(single_data, image_root, "Single Object")
check_missing_frames(multi_data, image_root, "Multi-Object")


🖼 Checking Missing Frames — Single Object
✅ All raw image frame files are present.

🖼 Checking Missing Frames — Multi-Object
✅ All raw image frame files are present.


In [36]:
import os
import json

# Paths
single_json_path = "output/single_object_boundingboxes_labels.json"
multi_json_path = "output/multi_object_boundingboxes_labels.json"
single_mask_root = "output/single_object_annotations"
multi_mask_root = "output/multi_object_annotations"

# Load JSONs
with open(single_json_path) as f:
    single_data = json.load(f)
with open(multi_json_path) as f:
    multi_data = json.load(f)

def check_missing_masks(data, mask_root, dataset_name):
    print(f"\n🎭 Checking Missing Masks — {dataset_name}")
    missing = []

    for seq, frames in data.items():
        seq_path = os.path.join(mask_root, seq)
        for frame_name in frames.keys():
            expected_mask_path = os.path.join(seq_path, frame_name)
            if not expected_mask_path.endswith(".png"):
                expected_mask_path += ".png"
            if not os.path.exists(expected_mask_path):
                missing.append(f"{seq}/{frame_name}.png")

    if missing:
        print("❌ Missing Mask Files:")
        for m in missing:
            print(f"• {m}")
    else:
        print("✅ All mask files are present.")

# Run checks
check_missing_masks(single_data, single_mask_root, "Single Object")
check_missing_masks(multi_data, multi_mask_root, "Multi-Object")


🎭 Checking Missing Masks — Single Object
✅ All mask files are present.

🎭 Checking Missing Masks — Multi-Object
✅ All mask files are present.


In [37]:
import json

# Paths to your JSON files
single_json_path = "output/single_object_boundingboxes_labels.json"
multi_json_path = "output/multi_object_boundingboxes_labels.json"

# Load JSONs
with open(single_json_path) as f:
    single_data = json.load(f)
with open(multi_json_path) as f:
    multi_data = json.load(f)

def validate_bounding_box_format(data, is_multi=False):
    errors = []
    for sequence, frames in data.items():
        for frame_name, annotations in frames.items():
            # In single-object case, get list of boxes
            if not is_multi:
                boxes = annotations.get("bbox", [])
                for box in boxes:
                    if not (isinstance(box, (list, tuple)) and len(box) == 4):
                        errors.append(f"{sequence}/{frame_name}: Invalid format → {box}")
                        continue
                    if not all(isinstance(x, (int, float)) and x >= 0 for x in box):
                        errors.append(f"{sequence}/{frame_name}: Non-numeric or negative values → {box}")
            else:
                # In multi-object case, check per object
                for obj_label, box in annotations.items():
                    if not (isinstance(box, (list, tuple)) and len(box) == 4):
                        errors.append(f"{sequence}/{frame_name}/{obj_label}: Invalid format → {box}")
                        continue
                    if not all(isinstance(x, (int, float)) and x >= 0 for x in box):
                        errors.append(f"{sequence}/{frame_name}/{obj_label}: Non-numeric or negative values → {box}")
    return errors

# Run validation
print("🔍 Checking Bounding Box Format — Single Object")
single_errors = validate_bounding_box_format(single_data, is_multi=False)
if single_errors:
    print("❌ Found format issues:")
    for err in single_errors:
        print(f"• {err}")
else:
    print("✅ All bounding boxes are well-formed.")

print("\n🔍 Checking Bounding Box Format — Multi-Object")
multi_errors = validate_bounding_box_format(multi_data, is_multi=True)
if multi_errors:
    print("❌ Found format issues:")
    for err in multi_errors:
        print(f"• {err}")
else:
    print("✅ All bounding boxes are well-formed.")

🔍 Checking Bounding Box Format — Single Object
✅ All bounding boxes are well-formed.

🔍 Checking Bounding Box Format — Multi-Object
✅ All bounding boxes are well-formed.


In [38]:
import os
import json
from PIL import Image

# Set paths
image_folder = "datasets/DAVIS_dataset/PNGImages/480p"
single_json_path = "output/single_object_boundingboxes_labels.json"
multi_json_path = "output/multi_object_boundingboxes_labels.json"

# Load JSONs
with open(single_json_path) as f:
    single_data = json.load(f)
with open(multi_json_path) as f:
    multi_data = json.load(f)

def check_out_of_bounds(data, image_folder, is_multi=False):
    out_of_bounds_errors = []

    for sequence_name, annotations in data.items():
        sequence_path = os.path.join(image_folder, sequence_name)
        if not os.path.isdir(sequence_path):
            continue  # Skip missing image sequences

        for frame_name, frame_data in annotations.items():
            image_path = os.path.join(sequence_path, frame_name)
            if not os.path.exists(image_path):
                continue  # Skip missing frames

            try:
                with Image.open(image_path) as img:
                    width, height = img.size
            except:
                continue  # Skip unreadable images

            # Get boxes based on JSON format
            if is_multi:
                # multi-object format: frame_name: { label: [x, y, w, h], ... }
                boxes = frame_data.values()
            else:
                # single-object format: frame_name: { "bbox": [[x, y, w, h], ...] }
                boxes = frame_data.get("bbox", [])

            for box in boxes:
                if not isinstance(box, list) or len(box) != 4:
                    continue  # Already validated earlier

                x, y, w, h = box
                if x < 0 or y < 0 or x + w > width or y + h > height:
                    out_of_bounds_errors.append(
                        f"{sequence_name}/{frame_name} | Box: {box} exceeds image size ({width}, {height})"
                    )

    return out_of_bounds_errors

# Run check
single_out_bounds = check_out_of_bounds(single_data, image_folder, is_multi=False)
multi_out_bounds = check_out_of_bounds(multi_data, image_folder, is_multi=True)

# Report
def print_bounds_report(name, errors):
    print(f"\n🚨 Out-of-Bounds Check — {name}")
    if errors:
        for err in errors:
            print(f"• {err}")
    else:
        print("✅ All bounding boxes are within image bounds.")

print_bounds_report("Single Object", single_out_bounds)
print_bounds_report("Multi-Object", multi_out_bounds)


🚨 Out-of-Bounds Check — Single Object
✅ All bounding boxes are within image bounds.

🚨 Out-of-Bounds Check — Multi-Object
✅ All bounding boxes are within image bounds.


In [13]:
import os
import json

# Paths
single_json_path = "output/single_object_boundingboxes_labels.json"
multi_json_path = "output/multi_object_boundingboxes_labels.json"
single_mask_root = "output/binary_single_object_masks"
multi_mask_root = "output/multi_object_label_masks"

# Load JSONs
with open(single_json_path) as f:
    single_data = json.load(f)
with open(multi_json_path) as f:
    multi_data = json.load(f)

def check_missing_masks(data, mask_root, dataset_name):
    print(f"\n🎭 Checking Missing Masks — {dataset_name}")
    missing = []

    for seq, frames in data.items():
        seq_path = os.path.join(mask_root, seq)
        for frame_name in frames.keys():
            expected_mask_path = os.path.join(seq_path, frame_name)
            if not expected_mask_path.endswith(".png"):
                expected_mask_path += ".png"
            if not os.path.exists(expected_mask_path):
                missing.append(f"{seq}/{frame_name}.png")

    if missing:
        print("❌ Missing Mask Files:")
        for m in missing:
            print(f"• {m}")
    else:
        print("✅ All mask files are present.")

# Run checks
check_missing_masks(single_data, single_mask_root, "Single Object")
check_missing_masks(multi_data, multi_mask_root, "Multi-Object")


🎭 Checking Missing Masks — Single Object
✅ All mask files are present.

🎭 Checking Missing Masks — Multi-Object
✅ All mask files are present.


In [ ]:
Using cv2.IMREAD_UNCHANGED preserves the original bit-depth and channel information in your label images, ensuring that the precise label values remain intact for accurate pixel-wise comparison with YOLO’s output.

In [12]:
import cv2
import numpy as np
from pathlib import Path

def get_unique_labels(image_path):
    """
    Loads a label image with cv2.IMREAD_UNCHANGED and returns the unique label values.
    
    Args:
        image_path (str or Path): Path to the label image.
        
    Returns:
        unique_labels (np.ndarray): Array of unique label values present in the image.
    """
    image_path = Path(image_path)
    img = cv2.imread(str(image_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"Could not load image: {image_path}")
        return None
    unique_labels = np.unique(img)
    return unique_labels

if __name__ == "__main__":
    # Specify the path to a single label image from your multi-object label masks.
    frame_path = "output/multi_object_label_masks/bmx-bumps/00020.png"
    labels = get_unique_labels(frame_path)
    if labels is not None:
        print(f"Unique label values in {frame_path}: {labels}")

Unique label values in output/multi_object_label_masks/bmx-bumps/00020.png: [0 1 2]


#### ! Using cv2.IMREAD_UNCHANGED preserves the original bit-depth and channel information in your label images, ensuring that the precise label values remain intact for accurate pixel-wise comparison with YOLO’s output !